<a href="https://colab.research.google.com/github/koperak/Advent-of-Code-2024/blob/main/AoC_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initialize support

In [1]:
import os
from pathlib import Path
from shutil import copyfile
import requests
from datetime import date
import logging
from argparse import ArgumentParser

logging.basicConfig(
    level=logging.INFO, format="%(levelname)-8s %(funcName)s: %(message)s",
)
logger = logging.getLogger(__name__)


BASE_DIR = "sample_data"
TEMPLATES_DIR = BASE_DIR + "/templates"


def setup_dir(day):
  newdir = Path(f"day{day:02d}")
  if not newdir.exists():
      os.mkdir(newdir)
      logger.info(f"created a new directory {newdir}")
  logger.info(f"done copying templates to {newdir}")


def get_input(day, year):

  from google.colab import userdata

  cookie = userdata.get('AOC_SESSION')

  if not cookie:
      logger.warning("no environment variable 'AOC_SESSION' found! skipping download")
      return

  url = f"https://adventofcode.com/{year}/day/{day}/input"
  dayfolder = f"day{day:02d}"
  if "input.txt" in os.listdir(dayfolder):
      logger.warning(f"{dayfolder}/input.txt already exists, skip download")
      return

  logger.info(f"download input from {url}... ")
  try:
      response = requests.get(
          url=url,
          cookies={"session": cookie},
          headers={"User-Agent": "https://github.com/davekch/aoc by dave-koch@web.de"},
      )
      if response.ok:
          data = response.text
          f = open(Path(dayfolder) / "input.txt", "w+")
          f.write(data.rstrip("\n"))
          f.close()
          logger.info("... done!")
      else:
          logger.error("server response not ok")
  except:
      logger.error("something went wrong")


Day 1 year 2024

Setup test input and test data

In [2]:
setup_dir(1)
get_input(1,2024)
input = open("day01/input.txt").read().splitlines()
test_input = """3   4
4   3
2   5
1   3
3   9
3   3"""
test_input = test_input.split("\n")

In [3]:
def prepare_input(input):
# prepare input
# return left and right list of integers
  _list_a, _list_b = zip(*[map(int, row.split("   ")) for row in input])
  return _list_a, _list_b

Part 1

In [4]:
left_list, right_list  = prepare_input(input)

In [5]:
sum(map(lambda x,y: abs(x - y), sorted(left_list), sorted(right_list)))

1258579

Part 2

In [6]:
from collections import OrderedDict, Counter
similarity_scores = Counter(right_list)

In [7]:
sum(map(lambda first_number: first_number * similarity_scores.get(first_number,0), left_list))

23981443

# Day 2 year 2024

## Setup test input and test data

In [8]:
setup_dir(2)
get_input(2,2024)
input = open("day02/input.txt").read().splitlines()
test_input = """7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9"""
test_input = test_input.split("\n")

In [9]:
def prepare_input(input):
# prepare input
# return left and right list of integers
  _list = [list(map(int, row.split())) for row in input]
  return _list

In [10]:
reports  = prepare_input(input)

## Part 1

In [11]:
differences = [[y - x for x, y in zip(report, report[1:])] for report in reports]

In [12]:
def is_increasing_or_decreasing_and_between2_3(difference):
  is_increasing = all(diff > 0 for diff in difference)
  is_decreasing = all(diff < 0 for diff in difference)
  is_between2_3 = all(1 <= abs(diff) <= 3 for diff in difference)
  return (is_increasing or is_decreasing) and is_between2_3

In [13]:
safe = 0
for difference in differences:
  if is_increasing_or_decreasing_and_between2_3(difference):
    safe = safe + 1

print(f'Part 1: {safe}')

Part 1: 257


##Part 2

In [14]:
def remove_one(input_list):
  """ input_list: list of integers
      retuns: yelds evety combinatnion of input_list with one element removed
  """
  for i in range(len(input_list)):
    yield input_list[:i] + input_list[i+1:]

In [15]:
def calc_differences(input_list):
  return [y - x for x, y in zip(input_list, input_list[1:])]

In [16]:
safe = 0
for report in reports:
  difference = calc_differences(report)
  if is_increasing_or_decreasing_and_between2_3(difference):
    safe = safe + 1
    print(f'Report: {report}, Safe: {safe}, Difference: {difference}')
  else:
    for one_removed in remove_one(report):
      difference = calc_differences(one_removed)
      if is_increasing_or_decreasing_and_between2_3(difference):
        safe = safe + 1
        print(f'Removed one: {one_removed}, Safe: {safe}, Difference: {difference}')
        break

print(f'Part 2: {safe}')

Removed one: [74, 76, 78, 79], Safe: 1, Difference: [2, 2, 1]
Removed one: [38, 40, 43, 44], Safe: 2, Difference: [2, 3, 1]
Removed one: [1, 2, 4, 6, 8, 9], Safe: 3, Difference: [1, 2, 2, 2, 1]
Removed one: [65, 68, 70, 72, 75, 76], Safe: 4, Difference: [3, 2, 2, 3, 1]
Removed one: [89, 91, 92, 93, 94], Safe: 5, Difference: [2, 1, 1, 1]
Removed one: [77, 78, 79, 82, 83], Safe: 6, Difference: [1, 1, 3, 1]
Removed one: [79, 82, 84, 85, 86], Safe: 7, Difference: [3, 2, 1, 1]
Removed one: [54, 57, 60, 61], Safe: 8, Difference: [3, 3, 1]
Removed one: [52, 54, 57, 60, 62, 64], Safe: 9, Difference: [2, 3, 3, 2, 2]
Removed one: [47, 49, 50, 53, 56, 59, 61], Safe: 10, Difference: [2, 1, 3, 3, 3, 2]
Removed one: [77, 78, 81, 83], Safe: 11, Difference: [1, 3, 2]
Removed one: [87, 86, 83, 82, 81, 78, 77], Safe: 12, Difference: [-1, -3, -1, -1, -3, -1]
Removed one: [55, 52, 51, 49], Safe: 13, Difference: [-3, -1, -2]
Removed one: [46, 43, 41, 39, 38, 37], Safe: 14, Difference: [-3, -2, -2, -1, -1]


# Day 3 year 2024

## Setup test input and test data

In [17]:
day = 3
setup_dir(day)
get_input(day,2024)
input = open(f"day0{day}/input.txt").read()
test_input = """xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"""

In [18]:
def prepare_input(input):
# prepare input
# return left and right list of integers
  return input

In [19]:
memory  = prepare_input(input)

## Part 1

In [20]:
import re

pattern = r'mul\((\d{1,3}),(\d{1,3})\)'

print(sum(x * y for x, y in [map(int, mul) for mul in re.findall(pattern, input)]))



180233229


##Part 2

In [21]:
new_pattern =  r"mul\(\d+,\d+\)|do\(\)|don't\(\)"

test_input = """xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"""
memory = prepare_input(input)

suma = 0
follow = True

for found in re.findall(new_pattern, input):
  match found:
      case "do()":
          follow = True
      case "don't()":
          follow = False
      case _:
          if follow:
              x, y = map(int, found[4:-1].split(','))
              suma += x * y

print(suma)

95411583
